In [ ]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

#dfAll = dfAll.replace({'':np.nan,0:np.nan})
dfAll = dfAll.reset_index()

In [2]:
#处理0这个比较异常的值,在trian set中有40条数据，在某些column上为0 ，需要处理，test_a中也有一条


(721, 7171)

In [ ]:
#number of NAN
dfAll['Total_NAN_Num']=dfAll[raw_predictors].T.isnull().sum().tolist()
for key,pro in procedure.items():
    dfAll[key+'_NAN_Num']=dfAll[pro].T.isnull().sum().tolist()

In [ ]:
#Count 计算特定值在变量中重复出现的次数
dfAll = dfAll.replace({np.nan:-99999999})
added = ['ID']
for var in raw_predictors:
    dfAll['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dfAll.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dfAll['cnt_'+var].ix[group.index] = count
dfAll[added].to_csv('../../Cache/Feat_cnt_col.csv',index=False)

added = ['ID']
for tool,pro in procedure.items():
    print('%.2f%%'%(float(t)*100/len(raw_predictors)))
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
dfAll[added].to_csv('../../Cache/Feat_cnt_tool.csv',index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
#percentile 计算某一个值在全集中的位置（百分制）
dfAll = dfAll.replace({:-99999999:np.nan})
added = ['ID']
for var in raw_predictors:
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)
dfAll[added].to_csv('../../Data/Feat_pcent_col.csv',index=False)
t = 0
added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
            for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
            
dfAll[added].to_csv('../../Data/Feat_pcent_tool.csv',index=False) 

In [ ]:
#Pivot 创建多组category variable 的两两交叉表,感觉只能用来计算分类型变量,好像现在直接可以用pandas解决
def get_pivottable(X_train, X_test, use='all',Feat_list):
    """
    Returns a list of dictionaries, one per feature in the
    basic data, containing cross-tabulated counts
    for each column and each value of the feature.
    """
    dictionaries = []
    if use == 'all':
        X = np.vstack((X_train, X_test))
        filename = "pivottable"
    elif use == 'train':
        X = X_train
        filename = "pivottable_train"
    else:
        X = X_test
        filename = "pivottable_test"
    for i in range(len(Feat_list)):
        dictionaries.append({'total': 0})
    try:
        with open("cache/%s.pkl" % filename, 'rb') as f:
            logger.debug("loading cross-tabulated data from cache")
            dictionaries = pickle.load(f)
    except IOError:
        logger.debug("no cache found, cross-tabulating data")
        for i, row in enumerate(X):
            for j in range(len(Feat_list)):
                dictionaries[j]['total'] += 1
                if row[j] not in dictionaries[j]:
                    dictionaries[j][row[j]] = {'total': 1}
                    for k, key in enumerate(Feat_list):
                        dictionaries[j][row[j]][key] = {row[k]: 1}
                else:
                    dictionaries[j][row[j]]['total'] += 1
                    for k, key in enumerate(Feat_list):
                        if row[k] not in dictionaries[j][row[j]][key]:
                            dictionaries[j][row[j]][key][row[k]] = 1
                        else:
                            dictionaries[j][row[j]][key][row[k]] += 1
        with open("cache/%s.pkl" % filename, 'wb') as f:
            pickle.dump(dictionaries, f, pickle.HIGHEST_PROTOCOL)

    return dictionaries

def creat_cate_corr(X_train, X_test, Feat_list=[var if 'TOOL' in var for var in raw_predictor]):
    features_train = []
    features_test = []
    dictionaries = get_pivottable(X_train[Feat_list], X_test[Feat_list],Feat_list=Feat_list)
    #dictionaries_train = get_pivottable(X_train[Feat_list], X_test[Feat_list], use='train',Feat_list=Feat_list)
    #dictionaries_test = get_pivottable(X_test[Feat_list], X_test[Feat_list], use='test',Feat_list=Feat_list)